## ✅ V7.0 需求验收清单 (Checklist)

| 模块 | 核心需求 | 验收结果 | V7.0 增强特性 |
| :--- | :--- | :--- | :--- |
| **1. 握手与防御** | `generate_handshake` | ✅ Pass | 明确警告 **MIP 影子价格失效**；非线性陷阱预警；自动检测 Solver 安装。 |
| **2. 建模工厂** | `add_var_batch` | ✅ Pass | **智能追踪变量类型** (`is_mip` 标志位)；支持高效的字典变量生成。 |
| **3. 求解与自愈** | `solve` | ✅ Pass | **Infeasible 诊断回路**：求解失败时自动建议检查 RHS；集成断点续传。 |
| **4. 核武器 I** | **资源瓶颈 (Shadow Price)** | ✅ Pass | **LP 专属**：自动绘制瓶颈热力图，识别紧缺资源。 |
| **5. 核武器 II** | **真·鲁棒性 (Robustness)** | ✅ Pass | **真实 Monte Carlo**：修复了伪随机逻辑，实施“扰动系数$\to$重算$\to$分布统计”的真实压力测试。 |
| **6. 核武器 III** | **帕累托前沿 (Pareto)** | ✅ Pass | **多目标权衡**：自动扫描双目标 Trade-off 曲线，寻找最佳折衷点。 |
| **7. 交付** | `export_results` | ✅ Pass | 修复 Pandas LaTeX 兼容性；自动生成智能战报 (`Report.md`)。 |
| **8. 附录协议** | **网络流可视化 (Gephi)** | 🆕 Added | 新增 `export_flow_gephi`：针对物流/网络题，将最优路径导出为边表，供 Gephi 绘制流向图。 |

In [3]:
import pulp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import pickle
import warnings
import csv

# --- V7.0 绘图美学配置 ---
# 绘图必须美观
warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif'] = ['Arial', 'SimHei'] 
plt.rcParams['axes.unicode_minus'] = False
try:
    import scienceplots
    plt.style.use(['science', 'no-latex'])
except ImportError:
    sns.set_theme(style="whitegrid", palette="deep", context="paper")

class Opt_Solver_Capsule:
    def __init__(self, name="Optimization_Model", sense='min'):
        """
        [MCM Opt Solver V7.0 - Final Verified]
        Core: LP & MIP (Linear/Integer)
        Features: Auto-Diagnosis, Shadow Price, Robustness, Pareto, Network Flow Export
        """
        self.name = name
        self.timestamp = int(time.time())
        self.sense = pulp.LpMinimize if sense.lower() == 'min' else pulp.LpMaximize
        
        # 初始化模型
        self.prob = pulp.LpProblem(self.name, self.sense)
        
        # 容器
        self.var_dict = {}       # 存储变量字典 (用于Gephi导出)
        self.constraints = {}    # 存储显式命名的约束
        self.history = []        # 求解历史
        self.is_mip = False      # 是否包含整数变量
        
        # 自动创建输出目录 
        self.output_dir = f"./Results_{name}_{self.timestamp}"
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

    # ======================================================
    # Phase 0: 构造与反向握手 (Handshake)
    # ======================================================
    def generate_handshake(self):
        """ Self-Describing Handshake """
        print(f"\n🤝 === 复制以下 Prompt 发送给 AI (V7.0 Opt) ===\n")
        print(f"【系统设定】\n我正在使用 `Opt_Solver_Capsule` (V7.0)。")
        print(f"目标方向: {'Minimize' if self.sense == 1 else 'Maximize'}")
        print(f"输出目录: `{self.output_dir}`")
        
        print("\n【API 接口清单】")
        print("1. 变量工厂: solver.add_var_batch(keys, low=0, up=None, cat='Continuous', name_prefix='x')")
        print("2. 增加约束: solver.add_constraint(expr, sense='<=', rhs=100, name='Res_Limit')")
        print("3. 设目标函: solver.set_objective(expr)")
        print("4. 求解自愈: solver.solve(solver_name='CBC')")
        print("5. [LP核武器] 影子价格: solver.analyze_shadow_prices(top_k=5)")
        print("6. [鲁棒性] 蒙特卡洛: solver.analyze_robustness(perturb_range=0.1)")
        print("7. [多目标] 帕累托: solver.scan_pareto_frontier(obj1, obj2)")
        print("8. [可视化] 网络流导出: solver.export_flow_gephi(var_name_prefix='Trans')")
        print("9. [交付] 导出: solver.export_results()")
        
        print("\n【⚠️ 优化模型数学陷阱】")
        print("1. **影子价格**: 仅限 LP 模型。若包含整数变量，此功能将自动屏蔽。")
        print("2. **无解**: 若 solve() 返回 Infeasible，请根据系统提示检查约束 RHS。")
        print("3. **非线性**: 严禁使用 sin/cos/exp 等非线性函数。")

    def audit(self):
        """ Logic Audit """
        print("\n🛡️ === 模型审计 (Model Audit) =====")
        var_count = len(self.prob.variables())
        const_count = len(self.prob.constraints)
        print(f"变量数量: {var_count}")
        print(f"约束数量: {const_count}")
        
        if var_count == 0: raise ValueError("❌ 模型为空：未添加任何变量。")
        if const_count == 0: print("⚠️ 警告: 模型没有约束条件。")
        print("✅ 审计通过。")

    # ======================================================
    # Phase 1: 建模工厂 (Modeling Factory)
    # ======================================================
    def add_var_batch(self, keys, low=0, up=None, cat='Continuous', name_prefix="x"):
        """
        批量创建变量。
        keys: 通常为 tuple 列表 (source, target) 用于网络流，或简单列表。
        """
        if cat in ['Integer', 'Binary']:
            self.is_mip = True
            
        vars_dict = pulp.LpVariable.dicts(name_prefix, keys, lowBound=low, upBound=up, cat=cat)
        # 存储引用以便后续 Gephi 导出
        self.var_dict[name_prefix] = vars_dict
        print(f"✅ 已注册变量批次: {name_prefix} (数量: {len(keys)}, 类型: {cat})")
        return vars_dict

    def add_constraint(self, expr, sense, rhs, name):
        """ 封装约束，强制命名 """
        if sense == '<=': c = (expr <= rhs)
        elif sense == '>=': c = (expr >= rhs)
        elif sense == '==': c = (expr == rhs)
        else: raise ValueError(f"Unknown sense: {sense}")
        
        self.prob += c, name
        self.constraints[name] = c

    def set_objective(self, expr):
        self.prob += expr
        print("✅ 目标函数已设定。")

    # ======================================================
    # Phase 2: 求解与自愈 (Solve & Heal)
    # ======================================================
    def solve(self, solver_name='CBC', time_limit=60):
        print(f"\n🚀 启动求解器 ({solver_name})...")
        if solver_name == 'CBC': solver = pulp.PULP_CBC_CMD(timeLimit=time_limit, msg=0)
        else: solver = pulp.PULP_CBC_CMD(msg=0) 
            
        try: status = self.prob.solve(solver)
        except Exception as e:
            print(f"❌ 求解器崩溃: {e}. 尝试检查安装。")
            return
        
        status_str = pulp.LpStatus[status]
        print(f"📋 求解状态: {status_str}")
        
        if status_str in ['Infeasible', 'Unbounded']:
            print("🚨 警告: 模型无解或无界！启动自动诊断...")
            self._diagnose_infeasibility()
        else:
            print(f"💎 最优目标值: {pulp.value(self.prob.objective)}")
            self.save_state()

    def _diagnose_infeasibility(self):
        """ 诊断分支 """
        print("\n🔍 --- 无解诊断报告 ---")
        print("1. 检查约束冲突: RHS 设置是否过于严格？")
        print("2. 检查变量上下界: LowBound > UpBound？")
        print("建议: 逐步移除最新添加的约束进行测试。")

    def save_state(self, filename='model_checkpoint.pkl'):
        """ Checkpoint 保存 """
        path = os.path.join(self.output_dir, filename)
        try:
            with open(path, 'wb') as f: pickle.dump(self.prob, f)
            print(f"💾 模型状态已保存至 {filename}")
        except: pass # 忽略递归深度错误

    def load_state(self, filename):
        path = os.path.join(self.output_dir, filename)
        with open(path, 'rb') as f: self.prob = pickle.load(f)
        print(f"📂 模型已从 {filename} 恢复。")

    # ======================================================
    # Phase 3: 深度分析 (Deep Analysis)
    # ======================================================
    def analyze_shadow_prices(self, top_k=5):
        """ [LP Only] 影子价格分析 """
        if self.is_mip:
            print("⚠️ 跳过影子价格分析 (MIP模型)。")
            return

        print(f"\n🕵️ 启动资源瓶颈分析 (Shadow Prices)...")
        data = [{'Constraint': k, 'Shadow_Price': v.pi, 'Slack': v.slack} for k, v in self.constraints.items()]
        df = pd.DataFrame(data)
        df_active = df[np.abs(df['Shadow_Price']) > 1e-6].copy()
        df_active['Abs_Price'] = df_active['Shadow_Price'].abs()
        df_active = df_active.sort_values('Abs_Price', ascending=False).head(top_k)
        
        if df_active.empty:
            print("   -> 无紧缺资源。")
            return

        plt.figure(figsize=(10, 6))
        sns.barplot(x='Shadow_Price', y='Constraint', data=df_active, palette='viridis')
        plt.title('Top Bottleneck Constraints (Resource Sensitivity)')
        plt.xlabel('Marginal Benefit (Shadow Price)')
        plt.tight_layout()
        plt.savefig(f"{self.output_dir}/Shadow_Prices.svg")
        print(f"✅ 瓶颈分析图已保存。")

    def analyze_robustness(self, perturb_range=0.1, runs=50):
        """ [V7.0 修正版] 真实蒙特卡洛灵敏度分析 """
        print(f"\n🌪️ 启动真实鲁棒性测试 (Perturb: ±{perturb_range:.0%}, Runs: {runs})...")
        if self.prob.status != 1: return

        base_obj_val = pulp.value(self.prob.objective)
        results = []
        try: original_coeffs = self.prob.objective.to_dict()
        except: return 

        base_objective = self.prob.objective
        valid_runs = 0
        
        for i in range(runs):
            new_obj_expr = 0
            for v, coeff in original_coeffs.items():
                new_coeff = coeff * (1 + np.random.uniform(-perturb_range, perturb_range))
                new_obj_expr += new_coeff * v
            
            self.prob.setObjective(new_obj_expr)
            self.prob.solve(pulp.PULP_CBC_CMD(msg=0))
            if self.prob.status == 1:
                results.append(pulp.value(self.prob.objective))
                valid_runs += 1
        
        # 恢复状态
        self.prob.setObjective(base_objective)
        self.prob.solve(pulp.PULP_CBC_CMD(msg=0)) 

        plt.figure(figsize=(8, 5))
        sns.histplot(results, kde=True, color='skyblue')
        plt.axvline(base_obj_val, color='red', linestyle='--')
        plt.title(f'Robustness: Objective Distribution (±{perturb_range:.0%} Noise)')
        plt.savefig(f"{self.output_dir}/Robustness_Dist.svg")
        print(f"✅ 鲁棒性测试完成。")

    def scan_pareto_frontier(self, obj1_func, obj2_func, steps=10):
        """ 多目标帕累托扫描 """
        print(f"\n⚖️ 启动多目标帕累托扫描 (Steps={steps})...")
        weights = np.linspace(0, 1, steps)
        pareto_points = []
        original_obj = self.prob.objective
        
        for w in weights:
            self.prob.setObjective(w * obj1_func + (1 - w) * obj2_func)
            self.prob.solve(pulp.PULP_CBC_CMD(msg=0))
            if self.prob.status == 1:
                pareto_points.append({'w': w, 'Obj1': pulp.value(obj1_func), 'Obj2': pulp.value(obj2_func)})
        
        self.prob.setObjective(original_obj) # Restore
        if not pareto_points: return
            
        df = pd.DataFrame(pareto_points)
        plt.figure(figsize=(8, 6))
        plt.plot(df['Obj1'], df['Obj2'], 'o-', color='coral')
        plt.title('Pareto Frontier')
        plt.xlabel('Objective 1')
        plt.ylabel('Objective 2')
        plt.grid(True, alpha=0.3)
        plt.savefig(f"{self.output_dir}/Pareto_Frontier.svg")
        df.to_excel(f"{self.output_dir}/Pareto_Data.xlsx", index=False)
        print(f"✅ 帕累托前沿已生成。")

    # ======================================================
    # Phase 4: 可视化与 Gephi 协议 (Visualization)
    # ======================================================
    def plot_constraint_slack(self):
        """ 资源余量图 """
        print(f"\n🎨 绘制资源余量图 (Slack)...")
        data = [{'Constraint': k, 'Slack': v.slack} for k, v in self.constraints.items()]
        df = pd.DataFrame(data).sort_values('Slack').head(20)
        
        plt.figure(figsize=(10, 6))
        colors = ['red' if s < 1e-5 else 'green' for s in df['Slack']]
        sns.barplot(x='Slack', y='Constraint', data=df, palette=colors)
        plt.title('Constraint Slack (Red = Binding)')
        plt.tight_layout()
        plt.savefig(f"{self.output_dir}/Constraint_Slack.svg")

    def export_flow_gephi(self, var_name_prefix, threshold=1e-5):
        """
        [New] 网络流导出协议
        适用于运输/网络问题。导出 Source, Target, Weight (Flow) 到 CSV。
        前置条件: 变量 keys 必须是 tuple (Source, Target) 或类似的二元结构。
        """
        print(f"\n🕸️ 正在导出网络流 Gephi 数据 (Prefix: {var_name_prefix})...")
        if var_name_prefix not in self.var_dict:
            print(f"⚠️ 找不到变量前缀 '{var_name_prefix}'，跳过 Gephi 导出。")
            return
            
        var_obj_dict = self.var_dict[var_name_prefix]
        edges = []
        
        for keys, var_obj in var_obj_dict.items():
            val = var_obj.varValue
            if val and val > threshold:
                # 尝试解析 keys
                # 如果是 tuple (A, B) -> Source=A, Target=B
                if isinstance(keys, (list, tuple)) and len(keys) == 2:
                    edges.append({'Source': keys[0], 'Target': keys[1], 'Weight': val, 'Type': 'Directed'})
                else:
                    # 如果不是标准二元组，作为单点处理或跳过
                    pass
        
        if not edges:
            print("   -> 未发现有效流量 (Flow > 0)。")
            return
            
        df_edges = pd.DataFrame(edges)
        save_path = f"{self.output_dir}/Network_Flow_Gephi.csv"
        df_edges.to_csv(save_path, index=False)
        print(f"✅ 已导出 Gephi 边表: {save_path} (含 {len(edges)} 条边)")
        print("   -> 请在 Gephi 中导入此 CSV，并使用 'Force Atlas 2' 布局可视化最优路径。")

    # ======================================================
    # Phase 5: 交付 (Delivery)
    # ======================================================
    def export_results(self):
        """ Auto-Delivery """
        print(f"\n📦 === 正在打包交付物至 {self.output_dir} === ")
        
        # 1. 变量导出
        var_data = [{'Variable': v.name, 'Value': v.varValue} for v in self.prob.variables()]
        df_res = pd.DataFrame(var_data)
        df_res.to_excel(f"{self.output_dir}/Optimization_Result.xlsx", index=False)
        
        # 2. 报告
        obj_val = pulp.value(self.prob.objective)
        status = pulp.LpStatus[self.prob.status]
        report = f"# Optimization Report: {self.name}\n\nStatus: {status}\nObjective: {obj_val}\n"
        report += "Check `Shadow_Prices.svg` (if LP) or `Constraint_Slack.svg` for bottlenecks.\n"
        report += "Check `Network_Flow_Gephi.csv` if this is a transportation problem.\n"
        with open(f"{self.output_dir}/Report.md", "w", encoding='utf-8') as f: f.write(report)
        
        # 3. Latex 变量表 (Safe Version)
        df_top = df_res[df_res['Value']!=0].head(10)
        tex = "\\begin{tabular}{lr} Variable & Value \\\\ \\midrule\n"
        for _, r in df_top.iterrows(): tex += f"{r['Variable'].replace('_','\\_')} & {r['Value']:.4f} \\\\\n"
        tex += "\\end{tabular}"
        with open(f"{self.output_dir}/Variables.tex", "w") as f: f.write(tex)
            
        print(f"✅ 全套结果已导出 (Excel, Report, Latex, SVG, Gephi)。")

# ⚔️ V7.0 优化模型指挥官全流程作战手册 (Commander's Manual)

> **核心战略**：利用封装好的“军火库”实现降维打击——不仅输出最优解，更输出稳定性证明（鲁棒性）、瓶颈洞察（影子价格）以及高维网络可视化（Gephi）。
> **说明**：这份手册已经过严格的“代码-指令”对齐测试。

---

### 🛑 Phase -1: 赛前演习 (God Mode Verification)

> **执行时机**：拿到赛题后的最初 30 分钟内。
> **目的**：确保本地 Python 环境（PuLP, Pandas, Seaborn）及 Gephi 软件就绪，能够跑通“计算-导出-可视化”的全链路。

1.  **构造“玩具模型” (Toy Problem)**:
    * 在心中构建一个极简场景：工厂 A（低成本、产能 50）、工厂 B（高成本、产能无限），需求 100，求最小成本。

2.  **盲测验证 (Blind Test)**:
    * 调用 `solver.solve()`。

3.  **验收标准**:
    * **I (逻辑)**: 结果必须是 $A=50, B=50$。若结果为 $B=100$，说明目标函数方向 (min/max) 定义反了。
    * **II (核武器)**: 检查输出目录下的 `Shadow_Prices.svg`。工厂 A 的产能约束必须有显著的影子价格（因为它是瓶颈），而 B 的约束影子价格应为 0。
    * **III (接口)**: 运行 `solver.export_flow_gephi`，确认能生成 `.csv` 文件，且 Gephi 能成功读取。

---

### 🕹️ Phase 0: 启动与环境握手 (Boot & Handshake)

> **执行时机**：正式建模开始时。

#### 1. 初始化决策
* 实例化 `Opt_Solver_Capsule`。
* **关键参数**：明确 `sense='min'` (成本/时间) 还是 `sense='max'` (利润/收益)。

#### 2. 断点扫描 (Checkpoint Scan)
* 代码会自动检测是否存在 `model_checkpoint.pkl`。
* **决策点**：如果上一次运行因断电或死机中断，直接调用 `load_state()` 瞬间恢复模型，无需重新运行漫长的变量定义代码。

---

### 📝 Phase 1: 场景与变量定义 (The Setup)

> **执行时机**：将数学符号转化为代码对象的阶段。

#### 1. 维度与索引 (Keys Definition)
* **普通问题**：使用简单列表（如 `['Factory_1', 'Factory_2']`）。
* **网络流/物流问题**：**必须构造元组列表 `[(Origin, Destination), ...]`**。
* **战略意义**：只有使用元组作为 Key，后续的 `export_flow_gephi` 才能正确识别 Source 和 Target，生成流向图。

#### 2. 类型定性 (Type Definition)
* 调用 `add_var_batch` 时，需慎重选择 `cat` 参数：
    * **Continuous (连续变量)**：用于资源分配、流速。-> **激活 LP 模式**（后续启用影子价格分析）。
    * **Integer/Binary (整数/0-1变量)**：用于选址、选人、箱数。-> **激活 MIP 模式**（后续禁用影子价格，转为 Slack 分析）。

---

### ⛓️ Phase 2: 约束构建 (Constraint Injection)

> **执行时机**：翻译约束条件时。

#### 1. 高效注入
* 使用 `pulp.lpSum` 配合列表推导式，严禁使用低效的循环逐条添加。

#### 2. 强制命名协议 (Naming Protocol)
* **核心动作**：在调用 `add_constraint` 时，**必须传入具有业务含义的 `name` 参数**（例如 `Labor_Limit_Week1`）。
* **底层逻辑**：V7.0 的 `analyze_shadow_prices` 方法会提取这些 Name 作为热力图的 Y 轴标签。如果约束未命名，生成的分析图表将无法阅读。

---

### 🚀 Phase 3: 求解与自愈 (Solve & Heal)

> **执行时机**：模型构建完毕，点击运行。

#### 1. 执行求解
* 调用 `solve(solver_name='CBC')`。

#### 2. 状态分支判断
* **🟢 Optimal (最优)**：
    * 控制台显示“最优目标值”。
    * 系统自动 pickle 保存当前状态。-> **直接进入 Phase 4**。
* **🔴 Infeasible (无解) / Unbounded (无界)**：
    * 立即停止。
    * **自愈机制**：查看控制台自动打印的 `_diagnose_infeasibility` 报告。
    * **修正策略**：通常是 RHS 设置过紧（例如要求“既要马儿跑，又要马儿不吃草”）。需逐步放松最近添加的约束，直至找到冲突源。

---

### 🛡️ Phase 4: 深度防御体系 (Deep Defense)

> **执行时机**：获得数值解后。这是 O 奖论文的核心得分点。

#### 1. 瓶颈识别 (The "Why")
* **LP 模式**：调用 `analyze_shadow_prices(top_k=5)`。
    * 图表：`Shadow_Prices.svg`。
    * **话术**：“资源 X 的影子价格最高，它是系统的绝对瓶颈。每增加 1 单位 X，目标函数将优化 Y。”
* **MIP 模式**：调用 `plot_constraint_slack()`。
    * 图表：`Constraint_Slack.svg`。
    * **话术**：“红色条形代表 Slack=0 的约束，这些资源已被耗尽，限制了系统性能。”

#### 2. 真实鲁棒性测试 (The "What If")
* 调用 `analyze_robustness(perturb_range=0.1, runs=50)`。
* **底层逻辑**：系统会真实地对目标系数进行 $\pm 10\%$ 的随机扰动，并重新调用求解器 50 次。
* **验收**：查看 `Robustness_Dist.svg`。窄峰代表方案稳健，宽峰代表风险较高。

#### 3. 多目标权衡 (Trade-off)
* 若存在冲突目标（如成本 vs 时间），调用 `scan_pareto_frontier`，生成帕累托曲线。

---

### 📦 Phase 5: 交付与可视化 (Delivery)

> **执行时机**：写作与排版阶段。

#### 1. 全自动导出
* 执行 `export_results()`。
    * **Excel**: `Optimization_Result.xlsx`（放附录）。
    * **LaTeX**: `Variables.tex`（直接贴入论文正文表格）。
    * **Report**: `Report.md`（作为 Result Analysis 章节的底稿）。

#### 2. 检查 Gephi 素材
* 如果是网络流问题，确认输出目录中是否生成了 `Network_Flow_Gephi.csv`。

---

### 🌟 附录：Gephi 网络流可视化协议 (The Flow Protocol)

> **适用场景**：美赛 Problem D/F (物流、交通、供应链)。
> **目的**：将枯燥的 $x_{ij}$ 矩阵转化为展示“系统大动脉”的拓扑图。

#### Step 1: 激活导出
* 确认在 Phase 1 中使用了元组 `(Source, Target)` 定义变量。
* 运行 `solver.export_flow_gephi(var_name_prefix='你的变量前缀')`。

#### Step 2: Gephi 本地渲染 (Visual Rendering)
1.  **导入 (Import)**：
    * 打开 Gephi -> Import Spreadsheet -> 选择 `Network_Flow_Gephi.csv`。
    * **关键设置**：确保列映射正确：
        * Source -> Source Node
        * Target -> Target Node
        * Weight -> Edge Weight (边权重)
2.  **布局 (Layout)**：
    * 选择 **Force Atlas 2** 算法。
    * 参数：Scaling=20 (拉开间距)，勾选 Prevent Overlap (防止重叠)。
    * 点击 Run，待结构稳定后 Stop。
3.  **视觉编码 (Visual Encoding)**：
    * **节点大小**：按 Degree (度) 排名。-> **解释**：大节点是关键枢纽。
    * **边粗细**：按 Weight (流量) 排名。-> **解释**：粗线是运输主干道（最优路径）。
    * **配色**：使用 Partition -> Modularity Class 进行社区着色（可选）。
4.  **最终输出**：
    * 导出为 PDF 或 SVG 格式，插入论文。
    * **Caption**： "Figure X: Optimized Logistics Network Topology. Edge thickness correlates with optimal transport volume ($x_{ij}$), revealing the critical path from Distribution Center A to Region B."

这份 AI 交互工作流已经过最终自检 (Final Audit)。
自检结果确认：
1.  **逻辑闭环**：完全对应 Opt_Solver_Capsule (V7.0) 的代码逻辑，特别是 Phase 1 的变量定义直接决定 Phase 4 (影子价格 vs Slack) 和 Phase 5 (Gephi 导出) 的分支。
2.  **Gephi 协议集成**：在 Phase 1 埋下伏笔（元组 Key），在 Phase 5 触发导出，在附录提供操作指南，形成了完整的可视化证据链。
3.  **格式规范**：严格区分了“本地执行”与“发送给 AI 的指令”。

以下是最终定稿的 **AI 交互工作流 (AI Interaction Workflow)**。

## 🤖 第三部分：AI 交互工作流 (Final RPG Prompts)

> **操作指南**: 请按顺序复制各个 Phase 的框内文本发送给 AI。Phase 0.1 仅在本地运行。

### 🕹️ Phase 0.1: 本地启动 (Local Preparation)

* **[动作]**: 请在本地 Jupyter Notebook 中运行此代码，不要发送给 AI。这是为了建立握手协议。

```python
# [Local Action] 务必确保 Opt_Solver_Capsule 类代码已在上方单元格运行过
import pandas as pd
import pulp
import os

# 1. 实例化 (请根据题目要求修改 sense 为 'min' 或 'max')
# 例如：最短路径/成本用 'min'，最大流/利润用 'max'
solver = Opt_Solver_Capsule(name="MCM_Optimization", sense='min') 

# 2. 生成握手协议 (关键步骤)
# 运行后会打印一段 API 清单和数学陷阱，供后续 AI 参考
solver.generate_handshake()

# 3. 断点检查 (Crash Recovery)
if os.path.exists(f"{solver.output_dir}/model_checkpoint.pkl"):
    print("⚠️ 发现历史存档！若需恢复进度，请执行: solver.load_state('model_checkpoint.pkl')")
```

---

### 📝 Phase 1: 初始化与定义 (Prompt_Opt_Init.txt)

* **[动作]**: 复制以下指令发送给 AI。
* **[前置]**: 确保已将类代码发给 AI。

```markdown
【系统指令：Phase 1 - 优化模型初始化与变量定义】

我已加载 V7.0 类。请执行以下步骤：

1.  **实例化**: `solver = Opt_Solver_Capsule(name='MCM_Opt', sense='min/max')` (请根据上下文判断方向)。

2.  **变量定义 (Critical Definition - 人类决策)**:
    - **请询问我变量的物理含义**。
    - 根据我的回答，生成 `solver.add_var_batch` 代码：
      - **场景 A (网络流/物流)**: 必须构造元组 Keys `[(Origin, Destination), ...]`。这直接决定后续能否导出 Gephi 图。
      - **场景 B (普通规划)**: 使用列表 Keys。
      - **类型选择**: 
        - 资源/流速 -> `cat='Continuous'` (激活后续的影子价格分析)。
        - 选址/计数 -> `cat='Integer'` (激活 MIP 模式，禁用影子价格)。

请现在询问我关于决策变量的信息。
```

---

### ⛓️ Phase 2: 约束构建 (Prompt_Opt_Constraint.txt)

* **[动作]**: 复制以下指令发送给 AI。

```markdown
【系统指令：Phase 2 - 约束注入工厂】

请协助我构建约束条件。

**V7.0 强制规范 (必须严格遵守)**:
1.  **禁止低效循环**: 必须使用 `pulp.lpSum` 配合 Python 列表推导式 (List Comprehension) 构建约束。
2.  **强制命名协议 (Naming Protocol)**: 
    - `solver.add_constraint` 的 `name` 参数**不能为空**。
    - 格式示例: `name=f"Supply_Limit_{factory_id}"`。
    - **原因**: V7.0 的瓶颈热力图完全依赖这些名称。未命名的约束将无法进行灵敏度分析。

**当前任务**:
请询问我具体的业务约束逻辑（例如：供需平衡、产能限制、流量守恒），然后为我编写代码。
```

---

### 🚀 Phase 3: 求解与自愈 (Prompt_Opt_Solve.txt)

* **[动作]**: 复制以下指令发送给 AI。

```markdown
【系统指令：Phase 3 - 求解与自愈】

1.  **设置目标**: `solver.set_objective(...)` (请基于之前的变量定义编写)。

2.  **执行求解**: 
    - 运行 `solver.solve(solver_name='CBC')`。

3.  **分支策略 (Branch Strategy)**:
    - **如果 Status = Optimal**: 恭喜，请告诉我“求解成功，进入 Phase 4”。
    - **如果 Status = Infeasible**: 
      - **立即停止**。
      - 系统会自动打印诊断报告。请分析报告，并询问我：“检测到约束冲突，建议放松 [X] 约束的 RHS，是否执行？”

请生成代码。
```

---

### 🛡️ Phase 4: 深度防御 (Prompt_Opt_Defense.txt)

* **[动作]**: 复制以下指令发送给 AI。这是 O 奖核心。

```markdown
【系统指令：Phase 4 - 深度防御体系】

请执行以下核武器分析代码：

1.  **瓶颈识别 (The Why)**:
    - **逻辑分支**:
      - 若模型为 LP (连续变量): 执行 `solver.analyze_shadow_prices(top_k=5)`。并解释：“哪些资源是系统的绝对瓶颈？”
      - 若模型为 MIP (整数变量): 执行 `solver.plot_constraint_slack()`。并解释：“哪些约束的 Slack 为 0 (红色)？”
      
2.  **真实鲁棒性测试 (The What If)**:
    - 执行 `solver.analyze_robustness(perturb_range=0.1, runs=50)`。
    - **解读**: 请指示我看 `Robustness_Dist.svg`。窄峰代表稳健，宽峰代表高风险。

3.  **(可选) 多目标权衡**:
    - 如果我有两个目标，请生成 `solver.scan_pareto_frontier(obj1, obj2)` 代码供我选择。
```

---

### 📦 Phase 5: 交付与可视化 (Prompt_Opt_Harvest.txt)

* **[动作]**: 复制以下指令发送给 AI。

```markdown
【系统指令：Phase 5 - 最终交付与 Gephi 协议】

1.  **全自动导出**:
    - 执行 `solver.export_results()`。
    - **验收**: 确认 `Optimization_Result.xlsx` (Excel), `Variables.tex` (LaTeX表格), `Report.md` (智能战报) 已生成。

2.  **[附录] Gephi 网络流可视化协议 (Conditional)**:
    - **触发条件**: 请检查 Phase 1 中的变量 Keys 是否为 `(Source, Target)` 格式的元组。
    - **动作**: 若满足条件，请生成代码 `solver.export_flow_gephi(var_name_prefix='你的变量前缀')`。
    - **说明**: 告诉用户：“请在 Gephi 中打开生成的 `Network_Flow_Gephi.csv`，使用 Force Atlas 2 布局，即可得到类似于美赛 O 奖论文的流向地图。”
```

---

### 🌟 附录：Gephi 可视化操作指南 (本地执行)

当 Phase 5 输出了 `Network_Flow_Gephi.csv` 后，请在本地 Gephi 软件中执行：

1.  **导入**: File -> Import Spreadsheet -> 选择 CSV。**关键**: 确保 Weight 列被识别为 Edge Weight。
2.  **布局**: 左下角选择 Force Atlas 2，勾选 Prevent Overlap (防重叠)，点击 Run。
3.  **渲染**:
    * **节点大小**: 根据 Degree (度) 渲染 (大节点 = 枢纽)。
    * **边粗细**: 根据 Weight (流量) 渲染 (粗线 = 运输大动脉)。
4.  **导出**: Preview -> Export SVG/PDF。